In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets
import os
from PIL import Image
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

In [ ]:
torch.cuda.is_available()

In [ ]:
if torch.cuda.is_available():
  print(torch.cuda.get_device_name(0))
else:
  print("No GPU")

In [ ]:
pwd

# **Part 1 - Data Preprocessing**

In [ ]:
with Image.open('CATS_DOGS\\train\\DOG\\0.jpg') as im:
  display(im)

In [ ]:
path = 'CATS_DOGS\\train' #path right before the train test split
img_names =[] #creating an empty list to take in all the image names

for folder,subfolders,filenames in os.walk(path):
  for img in filenames:
    img_names.append(folder+'/'+img)

print('There are',len(img_names),'images')
print(img_names[-1])

In [ ]:
img_sizes = []
rejected = []

for item in img_names:
  try:
    with Image.open(item) as img:
      img_sizes.append(img.size)

  except:
    rejected.append(item)

print(len(img_sizes))
print(len(rejected))  

In [ ]:
# Setting some hyperparameters
batchSize = 64 # We set the size of the batch.
imageSize = 64 # We set the size of the generated images (64x64).

# Creating the transformations
transform = transforms.Compose([transforms.Resize((imageSize,imageSize)),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ]) # We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.

# Loading the dataset
dataset = datasets.ImageFolder(root = 'CATS_DOGS\\both', transform = transform) #have both the training and testing data in one folder to feed our Discriminator

dataloader = DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2) # We use dataLoader to get the images of the training set batch by batch. So now we're loading in the data from the CIFAR training Dataset, and we only load it in in batches of 10 rows at a time out of the 60000 images, with the rows being shuffled (random). Num_workers means how many subprocesses to use for data. so num_workers = 2 means we'll have 2 a parallel threads that will load the data, whereas num_workers=0 means that the data will be loaded in the main process. Use pin_memory = True if using GPU

In [ ]:
# Defining the weights_init function that takes as input a neural network m and that will initialize all its weights.
def weights_init(model):
    classname = model.__class__.__name__
    if classname.find('Conv') != -1:
        model.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        model.weight.data.normal_(1.0, 0.02)
        model.bias.data.fill_(0)

# **Part 2 - Creating the Generator**

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__() #super is a function that allows us to use Module's functions and variables, while also optimizing this inheritance process
        
        #Essentially, Sequential() sets up a sequence of neural network layers where each level can include Linear functions, activation functions, pooling layers, convolutional layers, drop out layers, etc. Using sequential combines the layers from a layers list so that it can be ordered properly as a NN, and it is useful for quickly adding in layers and activation functions. Modules will be added to it in the order they are passed in the constructor.
        #First we start with an inverse convolution (ConvTranspose2d), since the role of a generator is to generate fake images. Therefore since a CNN takes as images and returns an output vector classifying it, an inverse CNN will do the opposite by taking inputs a classifying vector and will create an image out of it. 
        #Second, we normalize the 512 feature maps which were outputed from the ConvTranspose2D
        #Third we apply the Rectified Linear Unit activation function
        #Repeat the 3 above steps until we output the 3 colour channels of the final, fake image
        #We finished with a hyperbolic tangent activation function, so that we don't break linearity, and so our values are between -1 to 1 (that's what the asymptopes of a hyperbolic tangent function is). We do this because it will then follow the same standard as the images in the dataset, since the created images of the Generator will be the inputs of the Discriminator along with the CIFAR10 photos. 
        self.main = nn.Sequential(
            nn.ConvTranspose2d(in_channels = 100, out_channels = 512, kernel_size = 4, stride = 1, padding = 0, bias = False),
            nn.BatchNorm2d(num_features = 512),
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels = 512, out_channels = 256, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(num_features = 256),
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels = 256, out_channels = 128, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(num_features = 128),
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels = 128, out_channels = 64, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(num_features = 64),
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels = 64, out_channels = 3, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.Tanh()          
            ) 
    #This next function is to create the forward propagations through our NN
    def forward(self, input):
      output = self.main(input) #forwarding through our main Sequential function to get the output
      return output

In [ ]:
netG = Generator()
netG.apply(weights_init) #initializing the weights of our discriminator

# **Part 3 - Creating the Discriminator**

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__() #super is a function that allows us to use Module's functions and variables, while also optimizing this inheritance process
        
        #Essentially, Sequential() sets up a sequence of neural network layers where each level can include Linear functions, activation functions, pooling layers, convolutional layers, drop out layers, etc. Using sequential combines the layers from a layers list so that it can be ordered properly as a NN, and it is useful for quickly adding in layers and activation functions. Modules will be added to it in the order they are passed in the constructor.
        #First we start with an convolution (Conv2d), since the role of a Discriminator is to take the images of real objects and output images from the Geneator, and distinguist them. As such, the parameters of the Generator will be the input of the Discriminator. Therefore, the CNN takes in images and returns an output vector classifying it
        #Second, we're using a Leaky ReLU as the activation function, with a negative slope of 0.2 (found thru experimentation), as it showed to perform better than a regular ReLU
        #Third we make the next convolutional layer
        #Fourth, we now normalize the 128 feature maps this time
        #Fifth, use the Leaky ReLU again
        #Repeat Steps 3 to 5 above steps until we output the 1, which will do the classifying
        #We finished with a sigmoid activation function, so that we don't break linearity, and our value will be either 0 or 1 to do the classification
        self.main = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.LeakyReLU(negative_slope = 0.2, inplace = True),
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(num_features = 128),
            nn.LeakyReLU(negative_slope = 0.2, inplace = True),
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(num_features = 256),
            nn.LeakyReLU(negative_slope = 0.2, inplace = True),
            nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.BatchNorm2d(num_features = 512),
            nn.LeakyReLU(negative_slope = 0.2, inplace = True),
            nn.Conv2d(in_channels = 512, out_channels = 1, kernel_size = 4, stride = 1, padding = 0, bias = False),
            nn.Sigmoid()       
            ) 
        
    #This next function is to create the forward propagations through our NN
    def forward(self, input):
      output = self.main(input) #forwarding through our main Sequential function to get the output
      return output.view(-1) #so recall we have to actually flatten the CNN before feeding it back into our Generator so it can compare the loss. So here, we're just taking the CNN which is 2 dimensions (cuz it's an image), and flattening it into 1 dimension vector using -1 which infers. This flattened result will complement the batch size, as ANNs and the Generator takes information in batches and it's dimensions will be (batch_size, flattened_CNN_vector)

In [ ]:
netD = Discriminator()
netD.apply(weights_init) #initializing the weights of our discriminator

# **Part 4 - Training the GANs**

In [ ]:
#example
ran = torch.rand(2,3,1,2)
random_tensor_ex = (ran)
print(random_tensor_ex)
random_variable_ex = Variable(ran)
print(random_variable_ex)
print(random_tensor_ex.size()[0])

In [ ]:
criterion = nn.BCELoss() #so our loss measurement will be based off Binary Cross Entrpy Loss, since this is a mutually exclusive binary answer (either a yes or no, cannot be both)

optimizer_D = torch.optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999)) #model parameters are just the fully connected layers and we are using Adam optimizer to optimize them. Betas are coefficients used for computing running averages of gradient and its square
optimizer_G = torch.optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999)) #model parameters are just the fully connected layers and we are using Adam optimizer to optimize them. Betas are coefficients used for computing running averages of gradient and its square

In [ ]:
#WITH GPU
if torch.cuda.is_available():
  netD = netD.cuda()
  netG = netG.cuda()

  import time #Gonna keep track of how long it takes to train our model

  start_time = time.time()

  #This for loop trains our NN
  for epoch in range(100):
    for i, data in enumerate(dataloader, 0):
      #i = batches and enumerate just counts what batch number we're on with i (starting from 0), and basically using tuple unpacking to get the actual image data
      #data is essentially 2 elements in a tuple, which are (X_train, y_train) of the real images

      #Step 1: Updating the weights of the neural network of the Discriminator
      netD.zero_grad() #initialzing the gradient on the model
      
      #1.1: Training the Discriminator with a real image from the dataset
      real, _ = data #doing tuple unpacking of data, but only the first element, as saying _ indicates we don't carea bout the second element. Real is the input batches of data, basically X_train of real images
      input = Variable(real) #converting our real images into a torch variable, which consist of the tensor data as well as the gradient. Note that we already converted the image data into a PyTorch Tensor above by using transform
      target_real = Variable(torch.ones(input.size()[0])) #target is a torch matrix of all ones, since the answer is always correct = 1 for real images. Grabbing index 0 just shows how many batches there are, since the tensor is (batch_size, tuples within each batch, elements within tuple). The batch size in this case is 64, as made at the beginning. See example below
      target_fake = Variable(torch.zeros(input.size()[0]))#target is a torch matrix of all zeros, since the answer is always correct = 0 for fake images. Grabbing index 0 just shows how many batches there are, since the tensor is (batch_size, tuples within each batch, elements within tuple). The batch size in this case is 64, as made at the beginning. See example below
      target_gen = Variable(torch.ones(input.size()[0])) #so we plan on comparing the outputs of the Discriminator with the target. As such, this means that all the images that the Discriminator accepted as correct, it thought it's target was 1 even if it's incorrect. Therefore, the Generator will only receive data from the Discriminator that it thought was 1, since that is the image we are trying to re-create properly.

      input = input.cuda()
      target_real = target_real.cuda()

      output = netD.forward(input) #getting the output by forward propagating our netD class
      loss_realD = criterion(output, target_real) #calculating the loss error

      #1.2: Training the Discriminator with a fake image from the Generator
      noise = torch.randn(input.size()[0], 100, 1, 1) #(batch size = 64, number of elements per batch (which is 100, because that's what the input of the Generator is), 1x1 item per element). See example below. Here we're comparing noise to the Discriminator to initialize the learning process
      noise = Variable(noise) #converting our nosie into a torch variable, so that it has both the noise data and the gradient

      noise = noise.cuda()
      target_fake = target_fake.cuda()

      fake = netG.forward(noise) # so we took the noise, fed it through our netG to create noise images
      output = netD.forward(fake.detach()) #now we're taking the noise images from the Generator and feeding it through our Discriminator to train on if the image is a dog or not. We used .detach() to save memory, since fake is a torch variable meaning it has gradient data. We are simple detaching this gradient data from the Generator, since it is not needed to forward propagate through the Discriminator
      loss_fakeD = criterion(output, target_fake) #calculating the loss error

      #1.3: Backpropagating the total error
      loss_D = loss_realD + loss_fakeD #getting total loss
      loss_D.backward() #doing backpropagation off the loss function
      optimizer_D.step() #using the Discriminator optimizer for the back propagation. So here we just care about the discriminator, because we're just trying to get it good at telling the difference between dogs and noise


      #Step 2: Updating the Weights of the neural network of the Generator
      netG.zero_grad() #initialzing the gradient on the model

      target_gen = target_gen.cuda()

      output = netD.forward(fake) #so here we're not using .detach(), because we want to keep the Generator gradient of the fake images, since we want to use it to train
      loss_G = criterion(output, target_gen)
      loss_G.backward() #doing backpropagation off the loss function
      optimizer_G.step() #using the Generator optimizer for the back propagation


      #Step 3: Printing losses and saving the real images/generated images
      if i % 700 == 0:
        print(f'Epoch: {epoch}    Step: {i}     Loss_D: {loss_D.item():10.8f}   Loss_G: {loss_G.item():10.8f}')

      if i % 100 == 0:
        vutils.save_image(real, 'Results\\real_samples.png', normalize = True) #saving the image every 100 batches using torchvision.utils
        fake = netG(noise)
        vutils.save_image(fake.data, 'Results\\fake_samples_epoch_%03d.png' %(epoch), normalize = True) #saving the image every 100 batches using torchvision.utils

  print(f'Training took {(time.time() - start_time)/60} minutes')
else:
  print('Using CPU, run next cell')

In [ ]:
#NO GPU
if torch.cuda.is_available():
  print('Using GPU, run cell above')
else:
  import time #Gonna keep track of how long it takes to train our model

  start_time = time.time()

  #This for loop trains our NN
  #This for loop trains our NN
  for epoch in range(480,500):
    for i, data in enumerate(dataloader, 0):
      #i = batches and enumerate just counts what batch number we're on with i (starting from 0), and basically using tuple unpacking to get the actual image data
      #data is essentially 2 elements in a tuple, which are (X_train, y_train) of the real images

      #Step 1: Updating the weights of the neural network of the Discriminator
      netD.zero_grad() #initialzing the gradient on the model
      
      #1.1: Training the Discriminator with a real image from the dataset
      real, _ = data #doing tuple unpacking of data, but only the first element, as saying _ indicates we don't carea bout the second element. Real is the input batches of data, basically X_train of real images
      input = Variable(real) #converting our real images into a torch variable, which consist of the tensor data as well as the gradient. Note that we already converted the image data into a PyTorch Tensor above by using transform
      target_real = Variable(torch.ones(input.size()[0])) #target is a torch matrix of all ones, since the answer is always correct = 1 for real images. Grabbing index 0 just shows how many batches there are, since the tensor is (batch_size, tuples within each batch, elements within tuple). The batch size in this case is 64, as made at the beginning. See example below
      target_fake = Variable(torch.zeros(input.size()[0]))#target is a torch matrix of all zeros, since the answer is always correct = 0 for fake images. Grabbing index 0 just shows how many batches there are, since the tensor is (batch_size, tuples within each batch, elements within tuple). The batch size in this case is 64, as made at the beginning. See example below
      target_gen = Variable(torch.ones(input.size()[0])) #so we plan on comparing the outputs of the Discriminator with the target. As such, this means that all the images that the Discriminator accepted as correct, it thought it's target was 1 even if it's incorrect. Therefore, the Generator will only receive data from the Discriminator that it thought was 1, since that is the image we are trying to re-create properly.

      output = netD.forward(input) #getting the output by forward propagating our netD class
      loss_realD = criterion(output, target_real) #calculating the loss error

      #1.2: Training the Discriminator with a fake image from the Generator
      noise = torch.randn(input.size()[0], 100, 1, 1) #(batch size = 64, number of elements per batch (which is 100, because that's what the input of the Generator is), 1x1 item per element). See example below. Here we're comparing noise to the Discriminator to initialize the learning process
      noise = Variable(noise) #converting our nosie into a torch variable, so that it has both the noise data and the gradient

      fake = netG.forward(noise) # so we took the noise, fed it through our netG to create noise images
      output = netD.forward(fake.detach()) #now we're taking the noise images from the Generator and feeding it through our Discriminator to train on if the image is a dog or not. We used .detach() to save memory, since fake is a torch variable meaning it has gradient data. We are simple detaching this gradient data from the Generator, since it is not needed to forward propagate through the Discriminator
      loss_fakeD = criterion(output, target_fake) #calculating the loss error

      #1.3: Backpropagating the total error
      loss_D = loss_realD + loss_fakeD #getting total loss
      loss_D.backward() #doing backpropagation off the loss function
      optimizer_D.step() #using the Discriminator optimizer for the back propagation. So here we just care about the discriminator, because we're just trying to get it good at telling the difference between dogs and noise


      #Step 2: Updating the Weights of the neural network of the Generator
      netG.zero_grad() #initialzing the gradient on the model

      output = netD.forward(fake) #so here we're not using .detach(), because we want to keep the Generator gradient of the fake images, since we want to use it to train
      loss_G = criterion(output, target_gen)
      loss_G.backward() #doing backpropagation off the loss function
      optimizer_G.step() #using the Generator optimizer for the back propagation
    

      #Step 3: Printing losses and saving the real images/generated images
      if i % 100 == 0:
        print(f'Epoch: {epoch:{20}}    Step: {i:{20}}     Loss_D: {loss_D.item():{30}}   Loss_G: {loss_G.item():{30}}    Time: {(time.time() - start_time)/60:->{30}} minutes')
        vutils.save_image(real, 'Results\\real_samples.png', normalize = True) #saving the image every 100 batches using torchvision.utils
        fake = netG(noise)
        vutils.save_image(fake.data, 'Results\\fake_samples_epoch_%03d.png' %(epoch), normalize = True) #saving the image every 100 batches using torchvision.utils

  print(f'Training took {(time.time() - start_time)/60} minutes')

In [ ]:
torch.save(netD.state_dict(), 'Discriminator.pt')
torch.save(netG.state_dict(), 'Generator.pt')
torch.save(netD.state_dict(),'Discriminator.net')
torch.save(netD.state_dict(),'Generator.net')